# Parameter optimization

In [1]:
import sys
sys.path.append('..')

import numpy as np
import pandas as pd
from glob import glob
from optimizer import Optimizer
from os import environ

# Set environment variable
environ["ENV"] = "optimize"

from config.config import ConfigFactory

pd.set_option('display.max_columns', 500)

ttype = 'sell'
pattern = ['Pattern', 'Trend']
indicator_list = pattern
indicator_list_higher = pattern

work_timeframe = '15m'
higher_timeframe = '1h'
opt_limit = 2000
load = False

print(f'Timeframe is {higher_timeframe}, trade type is {ttype}')

# Get configs
configs = ConfigFactory.factory(environ).configs
configs['Indicator_list'] = pattern
configs['Higher_TF_indicator_list'] = indicator_list_higher
configs['Timeframes']['work_timeframe'] = work_timeframe
configs['Timeframes']['higher_timeframe'] = higher_timeframe

optim_dict = {
                'Pattern': {
                            'use_vol': [0], 
                            'window_low_bound': [1], 
                            'window_high_bound': [5],
                            'first_candle': [0.9], # np.arange(0.5, 1, 0.05),
                            'second_candle': [0.9], # np.arange(0.5, 1, 0.05),
                            'third_candle': [0.5]
                            },
                'Trend': {
                            'timeperiod': [90], # np.arange(2, 102, 2),
                            'low_bound': [0]
                            }
             }

opt = Optimizer(pattern, optim_dict, **configs)
stat = opt.optimize(pattern, ttype, opt_limit, load)

# stat_list = glob(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}*')
# if not stat_list:
#     stat.to_pickle(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}.pkl')
# else:
#     stat.to_pickle(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}_{len(stat_list)}.pkl')


Timeframe is 1h, trade type is sell
Number of combinations is 36


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 36/36 [02:51<00:00,  4.76s/it]


# Check local statistics

In [8]:
# candles
e_ratio_cols = [f'e_ratio_{lag + 1}' for lag in range(24)]
pct_price_diff_cols = [f'pct_price_diff_{lag + 1}' for lag in range(24)]
pct_price_diff_cols = [f'pct_price_diff_{lag + 1}' for lag in range(24)]

stat['e_ratio_avg'] = stat[[f'e_ratio_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['mar_avg'] = stat[[f'pct_price_diff_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['e_ratio_rank'] = (stat['e_ratio_avg'] - 1) * stat['forecasts_num']
stat['mar_rank'] = stat['mar_avg'] * stat['forecasts_num']

stat = stat[[c for c in stat.columns if c not in e_ratio_cols + pct_price_diff_cols] + e_ratio_cols + 
            pct_price_diff_cols].sort_values('mar_avg')
stat.head(20)

,pattern,Pattern_use_vol,Pattern_window_low_bound,Pattern_window_high_bound,Pattern_first_candle,Pattern_second_candle,Pattern_third_candle,Trend_timeperiod,Trend_low_bound,forecasts_num,e_ratio_avg,mar_avg,e_ratio_rank,mar_rank,e_ratio_1,e_ratio_2,e_ratio_3,e_ratio_4,e_ratio_5,e_ratio_6,e_ratio_7,e_ratio_8,e_ratio_9,e_ratio_10,e_ratio_11,e_ratio_12,e_ratio_13,e_ratio_14,e_ratio_15,e_ratio_16,e_ratio_17,e_ratio_18,e_ratio_19,e_ratio_20,e_ratio_21,e_ratio_22,e_ratio_23,e_ratio_24,pct_price_diff_1,pct_price_diff_2,pct_price_diff_3,pct_price_diff_4,pct_price_diff_5,pct_price_diff_6,pct_price_diff_7,pct_price_diff_8,pct_price_diff_9,pct_price_diff_10,pct_price_diff_11,pct_price_diff_12,pct_price_diff_13,pct_price_diff_14,pct_price_diff_15,pct_price_diff_16,pct_price_diff_17,pct_price_diff_18,pct_price_diff_19,pct_price_diff_20,pct_price_diff_21,pct_price_diff_22,pct_price_diff_23,pct_price_diff_24
99,Pattern_Trend,0,0,5,0.95,0.95,0.5,28,0,90,0.797008,-0.430000,-18.269250,-38.700000,0.8751,0.8297,0.8902,0.8586,0.8175,0.7915,0.8175,0.8829,0.9151,0.9219,0.8787,0.8015,0.7641,0.7384,0.7373,0.7380,0.7184,0.7352,0.7546,0.7507,0.7501,0.7272,0.7257,0.7083,-0.04,-0.09,-0.13,-0.16,-0.21,-0.26,-0.31,-0.37,-0.42,-0.46,-0.50,-0.53,-0.56,-0.57,-0.58,-0.58,-0.58,-0.59,-0.59,-0.60,-0.60,-0.55,-0.54,-0.50
98,Pattern_Trend,0,0,5,0.95,0.90,0.5,28,0,106,0.738996,-0.425833,-27.666442,-45.138333,0.8585,0.7964,0.8315,0.8164,0.7743,0.7252,0.7370,0.8017,0.7872,0.7957,0.7672,0.7166,0.6925,0.6792,0.6878,0.6902,0.6800,0.6966,0.7121,0.7100,0.7099,0.6918,0.6905,0.6876,-0.04,-0.09,-0.12,-0.15,-0.20,-0.23,-0.28,-0.36,-0.40,-0.45,-0.49,-0.52,-0.55,-0.56,-0.58,-0.58,-0.58,-0.59,-0.60,-0.61,-0.61,-0.56,-0.55,-0.52
89,Pattern_Trend,0,0,5,0.90,0.95,0.5,28,0,105,0.836496,-0.394167,-17.167938,-41.387500,0.8648,0.8440,0.9407,0.9175,0.8825,0.8595,0.8845,0.9451,0.9611,0.9647,0.9104,0.8359,0.8012,0.7754,0.7643,0.7773,0.7540,0.7675,0.7847,0.7810,0.7815,0.7592,0.7652,0.7539,-0.04,-0.08,-0.12,-0.15,-0.20,-0.24,-0.30,-0.35,-0.40,-0.44,-0.48,-0.50,-0.53,-0.53,-0.54,-0.53,-0.52,-0.53,-0.53,-0.53,-0.53,-0.48,-0.47,-0.44
88,Pattern_Trend,0,0,5,0.90,0.90,0.5,28,0,124,0.760000,-0.393750,-29.760000,-48.825000,0.8649,0.8125,0.8301,0.8309,0.7935,0.7524,0.7645,0.8395,0.8189,0.8308,0.7943,0.7352,0.7112,0.6986,0.6973,0.7095,0.7036,0.7174,0.7308,0.7288,0.7299,0.7118,0.7151,0.7185,-0.04,-0.08,-0.12,-0.15,-0.19,-0.22,-0.27,-0.33,-0.37,-0.42,-0.46,-0.49,-0.51,-0.52,-0.53,-0.53,-0.53,-0.54,-0.55,-0.55,-0.56,-0.51,-0.50,-0.48
97,Pattern_Trend,0,0,5,0.95,0.85,0.5,28,0,126,0.769733,-0.302083,-29.013600,-38.062500,0.9165,0.8754,0.8847,0.8857,0.8420,0.7881,0.7975,0.8469,0.8434,0.8432,0.8082,0.7549,0.7231,0.7082,0.7001,0.7010,0.6931,0.7059,0.7163,0.7112,0.7116,0.6932,0.6626,0.6608,-0.03,-0.07,-0.10,-0.12,-0.16,-0.19,-0.22,-0.28,-0.32,-0.35,-0.38,-0.40,-0.41,-0.41,-0.42,-0.41,-0.40,-0.41,-0.41,-0.41,-0.40,-0.35,-0.32,-0.28
59,Pattern_Trend,0,0,5,0.75,0.95,0.5,28,0,175,0.868012,-0.300000,-23.097813,-52.500000,0.8063,0.7651,0.8939,0.8834,0.8679,0.8470,0.8632,0.9158,0.9559,0.9522,0.9135,0.8777,0.8642,0.8250,0.8209,0.8363,0.8492,0.8574,0.8832,0.8897,0.8855,0.8725,0.8574,0.8491,-0.02,-0.04,-0.06,-0.08,-0.10,-0.13,-0.17,-0.20,-0.24,-0.27,-0.31,-0.33,-0.36,-0.37,-0.39,-0.40,-0.42,-0.44,-0.46,-0.48,-0.49,-0.48,-0.48,-0.48
49,Pattern_Trend,0,0,5,0.70,0.95,0.5,28,0,196,0.896088,-0.293333,-20.366850,-57.493333,0.8183,0.7822,0.8624,0.8441,0.8430,0.8344,0.8465,0.9344,0.9748,0.9941,0.9639,0.9310,0.9188,0.8810,0.8782,0.8951,0.9023,0.9076,0.9240,0.9252,0.9307,0.9181,0.9034,0.8926,-0.02,-0.03,-0.05,-0.06,-0.08,-0.10,-0.13,-0.17,-0.20,-0.24,-0.28,-0.30,-0.34,-0.36,-0.38,-0.40,-0.42,-0.45,-0.47,-0.49,-0.51,-0.51,-0.52,-0.53
96,Pattern_Trend,0,0,5,0.95,0.80,0.5,28,0,147,0.769771,-0.292917,-33.843688,-43.058750,0.9189,0.8407,0.8368,0.9111,0.8744,0.8129,0.8148,0.8487,0.8433,0.8361,0.7912,0.7430,0.7137,0.6990,0.6908,0.6914,0.6960,0.7106,0.7210,0.7182,0.7129,0.6990,0.6732,0.6768,-0.03,-0.06,-0.10,-0.13,-0.17,-0.20,-0.23,-0.2

In [3]:
# LR timeperiod
e_ratio_cols = [f'e_ratio_{lag + 1}' for lag in range(24)]
pct_price_diff_cols = [f'pct_price_diff_{lag + 1}' for lag in range(24)]
pct_price_diff_cols = [f'pct_price_diff_{lag + 1}' for lag in range(24)]

stat['e_ratio_avg'] = stat[[f'e_ratio_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['mar_avg'] = stat[[f'pct_price_diff_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['e_ratio_rank'] = (stat['e_ratio_avg'] - 1) * stat['forecasts_num']
stat['mar_rank'] = stat['mar_avg'] * stat['forecasts_num']

stat = stat[[c for c in stat.columns if c not in e_ratio_cols + pct_price_diff_cols] + e_ratio_cols + 
            pct_price_diff_cols].sort_values('mar_avg')
stat.head(20)

,pattern,Pattern_use_vol,Pattern_window_low_bound,Pattern_window_high_bound,Pattern_first_candle,Pattern_second_candle,Pattern_third_candle,Trend_timeperiod,Trend_low_bound,forecasts_num,e_ratio_avg,mar_avg,e_ratio_rank,mar_rank,e_ratio_1,e_ratio_2,e_ratio_3,e_ratio_4,e_ratio_5,e_ratio_6,e_ratio_7,e_ratio_8,e_ratio_9,e_ratio_10,e_ratio_11,e_ratio_12,e_ratio_13,e_ratio_14,e_ratio_15,e_ratio_16,e_ratio_17,e_ratio_18,e_ratio_19,e_ratio_20,e_ratio_21,e_ratio_22,e_ratio_23,e_ratio_24,pct_price_diff_1,pct_price_diff_2,pct_price_diff_3,pct_price_diff_4,pct_price_diff_5,pct_price_diff_6,pct_price_diff_7,pct_price_diff_8,pct_price_diff_9,pct_price_diff_10,pct_price_diff_11,pct_price_diff_12,pct_price_diff_13,pct_price_diff_14,pct_price_diff_15,pct_price_diff_16,pct_price_diff_17,pct_price_diff_18,pct_price_diff_19,pct_price_diff_20,pct_price_diff_21,pct_price_diff_22,pct_price_diff_23,pct_price_diff_24
49,Pattern_Trend,0,1,5,0.9,0.9,0.5,100,0,74,0.856871,-0.457917,-10.591558,-33.885833,0.8436,0.7932,0.8427,0.8470,0.7861,0.7690,0.8013,0.9060,0.9022,0.9363,0.9379,0.9082,0.8699,0.8426,0.8312,0.8514,0.8349,0.8367,0.8647,0.8514,0.8643,0.8886,0.8867,0.8690,-0.03,-0.06,-0.10,-0.13,-0.15,-0.18,-0.23,-0.28,-0.33,-0.39,-0.44,-0.48,-0.53,-0.57,-0.59,-0.62,-0.64,-0.67,-0.70,-0.72,-0.74,-0.80,-0.81,-0.80
44,Pattern_Trend,0,1,5,0.9,0.9,0.5,90,0,76,0.844608,-0.435833,-11.809767,-33.123333,0.8960,0.8225,0.8555,0.8410,0.7832,0.7668,0.7982,0.9002,0.8949,0.9283,0.9303,0.8931,0.8507,0.8204,0.8099,0.8198,0.8055,0.8076,0.8344,0.8223,0.8351,0.8577,0.8562,0.8410,-0.03,-0.07,-0.10,-0.13,-0.15,-0.18,-0.23,-0.28,-0.33,-0.38,-0.43,-0.47,-0.51,-0.55,-0.57,-0.59,-0.60,-0.63,-0.66,-0.67,-0.69,-0.74,-0.74,-0.73
47,Pattern_Trend,0,1,5,0.9,0.9,0.5,96,0,75,0.854471,-0.434167,-10.914688,-32.562500,0.8814,0.8163,0.8611,0.8549,0.7932,0.7757,0.8074,0.9109,0.9050,0.9386,0.9401,0.9035,0.8612,0.8348,0.8238,0.8336,0.8185,0.8204,0.8477,0.8351,0.8479,0.8718,0.8704,0.8540,-0.03,-0.06,-0.10,-0.12,-0.15,-0.17,-0.22,-0.27,-0.32,-0.38,-0.42,-0.46,-0.51,-0.54,-0.56,-0.58,-0.60,-0.63,-0.66,-0.68,-0.70,-0.75,-0.76,-0.75
46,Pattern_Trend,0,1,5,0.9,0.9,0.5,94,0,75,0.854471,-0.434167,-10.914688,-32.562500,0.8814,0.8163,0.8611,0.8549,0.7932,0.7757,0.8074,0.9109,0.9050,0.9386,0.9401,0.9035,0.8612,0.8348,0.8238,0.8336,0.8185,0.8204,0.8477,0.8351,0.8479,0.8718,0.8704,0.8540,-0.03,-0.06,-0.10,-0.12,-0.15,-0.17,-0.22,-0.27,-0.32,-0.38,-0.42,-0.46,-0.51,-0.54,-0.56,-0.58,-0.60,-0.63,-0.66,-0.68,-0.70,-0.75,-0.76,-0.75
45,Pattern_Trend,0,1,5,0.9,0.9,0.5,92,0,75,0.854471,-0.434167,-10.914688,-32.562500,0.8814,0.8163,0.8611,0.8549,0.7932,0.7757,0.8074,0.9109,0.9050,0.9386,0.9401,0.9035,0.8612,0.8348,0.8238,0.8336,0.8185,0.8204,0.8477,0.8351,0.8479,0.8718,0.8704,0.8540,-0.03,-0.06,-0.10,-0.12,-0.15,-0.17,-0.22,-0.27,-0.32,-0.38,-0.42,-0.46,-0.51,-0.54,-0.56,-0.58,-0.60,-0.63,-0.66,-0.68,-0.70,-0.75,-0.76,-0.75
48,Pattern_Trend,0,1,5,0.9,0.9,0.5,98,0,75,0.854471,-0.434167,-10.914688,-32.562500,0.8814,0.8163,0.8611,0.8549,0.7932,0.7757,0.8074,0.9109,0.9050,0.9386,0.9401,0.9035,0.8612,0.8348,0.8238,0.8336,0.8185,0.8204,0.8477,0.8351,0.8479,0.8718,0.8704,0.8540,-0.03,-0.06,-0.10,-0.12,-0.15,-0.17,-0.22,-0.27,-0.32,-0.38,-0.42,-0.46,-0.51,-0.54,-0.56,-0.58,-0.60,-0.63,-0.66,-0.68,-0.70,-0.75,-0.76,-0.75
2,Pattern_Trend,0,1,5,0.9,0.9,0.5,6,0,151,0.743287,-0.430417,-38.763587,-64.992917,0.9475,0.8926,0.8008,0.8089,0.8269,0.7870,0.7681,0.8127,0.7248,0.7499,0.7436,0.7037,0.6766,0.6625,0.6656,0.6854,0.6928,0.6895,0.6829,0.6949,0.7026,0.7058,0.7029,0.7109,-0.04,-0.08,-0.11,-0.14,-0.19,-0.23,-0.29,-0.35,-0.39,-0.43,-0.46,-0.48,-0.56,-0.57,-0.58,-0.58,-0.58,-0.60,-0.60,-0.62,-0.63,-0.61,-0.61,-0.60
3,Pattern_Trend,0,1,5,0.9,0.9,0.5,8,0,147,0.683246,-0.427083,-46.562862,-62.781250,0.9119,0.8383,0.7371,0.7608,0.7661,0.7383,0.7214,0.7670,0.6684,0.6893,0.6834,0.6484,0.6247,0.6114,0.6133,0.6236,0.6251,0.6205,0.6165,0.6291,0.6326,0.6209,0.6202,0.6296,-0.05,-0.09,-0.13,-0.16,-0.21,-0.25,-0.31,-0.38,-0.41,-0.45,-0.48,-0.5

In [2]:
# vol
e_ratio_cols = [f'e_ratio_{lag + 1}' for lag in range(24)]
pct_price_diff_cols = [f'pct_price_diff_{lag + 1}' for lag in range(24)]
pct_price_diff_cols = [f'pct_price_diff_{lag + 1}' for lag in range(24)]

stat['e_ratio_avg'] = stat[[f'e_ratio_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['mar_avg'] = stat[[f'pct_price_diff_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['e_ratio_rank'] = (stat['e_ratio_avg'] - 1) * stat['forecasts_num']
stat['mar_rank'] = stat['mar_avg'] * stat['forecasts_num']

stat = stat[[c for c in stat.columns if c not in e_ratio_cols + pct_price_diff_cols] + e_ratio_cols + 
            pct_price_diff_cols].sort_values('mar_avg', ascending=False)
stat.head(20)

,pattern,Pattern_use_vol,Pattern_window_low_bound,Pattern_window_high_bound,Pattern_first_candle,Pattern_second_candle,Pattern_third_candle,Trend_timeperiod,Trend_low_bound,forecasts_num,e_ratio_avg,mar_avg,e_ratio_rank,mar_rank,e_ratio_1,e_ratio_2,e_ratio_3,e_ratio_4,e_ratio_5,e_ratio_6,e_ratio_7,e_ratio_8,e_ratio_9,e_ratio_10,e_ratio_11,e_ratio_12,e_ratio_13,e_ratio_14,e_ratio_15,e_ratio_16,e_ratio_17,e_ratio_18,e_ratio_19,e_ratio_20,e_ratio_21,e_ratio_22,e_ratio_23,e_ratio_24,pct_price_diff_1,pct_price_diff_2,pct_price_diff_3,pct_price_diff_4,pct_price_diff_5,pct_price_diff_6,pct_price_diff_7,pct_price_diff_8,pct_price_diff_9,pct_price_diff_10,pct_price_diff_11,pct_price_diff_12,pct_price_diff_13,pct_price_diff_14,pct_price_diff_15,pct_price_diff_16,pct_price_diff_17,pct_price_diff_18,pct_price_diff_19,pct_price_diff_20,pct_price_diff_21,pct_price_diff_22,pct_price_diff_23,pct_price_diff_24
34,Pattern_Trend,1,3,6,0.9,0.9,0.5,90,0,2,0.251996,2.104167,-1.496008,4.208333,0.9859,1.0272,0.6054,0.2972,0.1786,0.1681,0.1681,0.1526,0.1526,0.1526,0.1526,0.1526,0.1526,0.1588,0.1543,0.1543,0.1543,0.1543,0.1543,0.1543,0.1543,0.1543,0.1543,0.1543,0.04,0.11,0.26,0.58,0.98,1.37,1.69,2.00,2.23,2.31,2.48,2.66,2.62,2.58,2.60,2.62,2.62,2.64,2.77,2.91,2.97,3.08,3.15,3.23
33,Pattern_Trend,1,3,5,0.9,0.9,0.5,90,0,2,0.251996,2.104167,-1.496008,4.208333,0.9859,1.0272,0.6054,0.2972,0.1786,0.1681,0.1681,0.1526,0.1526,0.1526,0.1526,0.1526,0.1526,0.1588,0.1543,0.1543,0.1543,0.1543,0.1543,0.1543,0.1543,0.1543,0.1543,0.1543,0.04,0.11,0.26,0.58,0.98,1.37,1.69,2.00,2.23,2.31,2.48,2.66,2.62,2.58,2.60,2.62,2.62,2.64,2.77,2.91,2.97,3.08,3.15,3.23
32,Pattern_Trend,1,3,4,0.9,0.9,0.5,90,0,2,0.251996,2.104167,-1.496008,4.208333,0.9859,1.0272,0.6054,0.2972,0.1786,0.1681,0.1681,0.1526,0.1526,0.1526,0.1526,0.1526,0.1526,0.1588,0.1543,0.1543,0.1543,0.1543,0.1543,0.1543,0.1543,0.1543,0.1543,0.1543,0.04,0.11,0.26,0.58,0.98,1.37,1.69,2.00,2.23,2.31,2.48,2.66,2.62,2.58,2.60,2.62,2.62,2.64,2.77,2.91,2.97,3.08,3.15,3.23
30,Pattern_Trend,1,2,6,0.9,0.9,0.5,90,0,2,0.251996,2.104167,-1.496008,4.208333,0.9859,1.0272,0.6054,0.2972,0.1786,0.1681,0.1681,0.1526,0.1526,0.1526,0.1526,0.1526,0.1526,0.1588,0.1543,0.1543,0.1543,0.1543,0.1543,0.1543,0.1543,0.1543,0.1543,0.1543,0.04,0.11,0.26,0.58,0.98,1.37,1.69,2.00,2.23,2.31,2.48,2.66,2.62,2.58,2.60,2.62,2.62,2.64,2.77,2.91,2.97,3.08,3.15,3.23
29,Pattern_Trend,1,2,5,0.9,0.9,0.5,90,0,2,0.251996,2.104167,-1.496008,4.208333,0.9859,1.0272,0.6054,0.2972,0.1786,0.1681,0.1681,0.1526,0.1526,0.1526,0.1526,0.1526,0.1526,0.1588,0.1543,0.1543,0.1543,0.1543,0.1543,0.1543,0.1543,0.1543,0.1543,0.1543,0.04,0.11,0.26,0.58,0.98,1.37,1.69,2.00,2.23,2.31,2.48,2.66,2.62,2.58,2.60,2.62,2.62,2.64,2.77,2.91,2.97,3.08,3.15,3.23
28,Pattern_Trend,1,2,4,0.9,0.9,0.5,90,0,2,0.251996,2.104167,-1.496008,4.208333,0.9859,1.0272,0.6054,0.2972,0.1786,0.1681,0.1681,0.1526,0.1526,0.1526,0.1526,0.1526,0.1526,0.1588,0.1543,0.1543,0.1543,0.1543,0.1543,0.1543,0.1543,0.1543,0.1543,0.1543,0.04,0.11,0.26,0.58,0.98,1.37,1.69,2.00,2.23,2.31,2.48,2.66,2.62,2.58,2.60,2.62,2.62,2.64,2.77,2.91,2.97,3.08,3.15,3.23
26,Pattern_Trend,1,1,6,0.9,0.9,0.5,90,0,2,0.251996,2.104167,-1.496008,4.208333,0.9859,1.0272,0.6054,0.2972,0.1786,0.1681,0.1681,0.1526,0.1526,0.1526,0.1526,0.1526,0.1526,0.1588,0.1543,0.1543,0.1543,0.1543,0.1543,0.1543,0.1543,0.1543,0.1543,0.1543,0.04,0.11,0.26,0.58,0.98,1.37,1.69,2.00,2.23,2.31,2.48,2.66,2.62,2.58,2.60,2.62,2.62,2.64,2.77,2.91,2.97,3.08,3.15,3.23
25,Pattern_Trend,1,1,5,0.9,0.9,0.5,90,0,2,0.251996,2.104167,-1.496008,4.208333,0.9859,1.0272,0.6054,0.2972,0.1786,0.1681,0.1681,0.1526,0.1526,0.1526,0.1526,0.1526,0.1526,0.1588,0.1543,0.1543,0.1543,0.1543,0.1543,0.1543,0.1543,0.1543,0.1543,0.1543,0.04,0.11,0.26,0.58,0.98,1.37,1.69,2.00,2.23,2.31,2.48,2.66,2.62,2.58,2.60,2.62,2.62,2.64,2.77,2.91,2.97,3.08,3.15,3.23
24,Pattern_Trend,1,1,4,0.9,0.9,0.5,90,0,2,0.251996,2.104167,-1.496008,4.208333,0.9859,1.0272,0.6054,0.2972,0.1786,0.1681,0.1681,0.1526,0.1526,0.1526,0.1526,0

In [4]:
# high/low bound
e_ratio_cols = [f'e_ratio_{lag + 1}' for lag in range(24)]
pct_price_diff_cols = [f'pct_price_diff_{lag + 1}' for lag in range(24)]
pct_price_diff_cols = [f'pct_price_diff_{lag + 1}' for lag in range(24)]

stat['e_ratio_avg'] = stat[[f'e_ratio_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['mar_avg'] = stat[[f'pct_price_diff_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['e_ratio_rank'] = (stat['e_ratio_avg'] - 1) * stat['forecasts_num']
stat['mar_rank'] = stat['mar_avg'] * stat['forecasts_num']

stat = stat[[c for c in stat.columns if c not in e_ratio_cols + pct_price_diff_cols] + e_ratio_cols + 
            pct_price_diff_cols].sort_values('mar_avg')
stat.head(20)

,pattern,Pattern_use_vol,Pattern_window_low_bound,Pattern_window_high_bound,Pattern_first_candle,Pattern_second_candle,Pattern_third_candle,Trend_timeperiod,Trend_low_bound,forecasts_num,e_ratio_avg,mar_avg,e_ratio_rank,mar_rank,e_ratio_1,e_ratio_2,e_ratio_3,e_ratio_4,e_ratio_5,e_ratio_6,e_ratio_7,e_ratio_8,e_ratio_9,e_ratio_10,e_ratio_11,e_ratio_12,e_ratio_13,e_ratio_14,e_ratio_15,e_ratio_16,e_ratio_17,e_ratio_18,e_ratio_19,e_ratio_20,e_ratio_21,e_ratio_22,e_ratio_23,e_ratio_24,pct_price_diff_1,pct_price_diff_2,pct_price_diff_3,pct_price_diff_4,pct_price_diff_5,pct_price_diff_6,pct_price_diff_7,pct_price_diff_8,pct_price_diff_9,pct_price_diff_10,pct_price_diff_11,pct_price_diff_12,pct_price_diff_13,pct_price_diff_14,pct_price_diff_15,pct_price_diff_16,pct_price_diff_17,pct_price_diff_18,pct_price_diff_19,pct_price_diff_20,pct_price_diff_21,pct_price_diff_22,pct_price_diff_23,pct_price_diff_24
9,Pattern_Trend,-1,3,5,0.9,0.9,0.5,90,0,47,0.801129,-0.646250,-9.346929,-30.373750,0.8660,0.9056,0.9502,0.8647,0.7507,0.7462,0.7777,0.9115,0.8954,0.8988,0.8757,0.8384,0.7748,0.7359,0.7366,0.7411,0.7087,0.7053,0.7449,0.7319,0.7315,0.7741,0.7894,0.7720,-0.05,-0.11,-0.17,-0.22,-0.26,-0.31,-0.38,-0.46,-0.52,-0.60,-0.66,-0.72,-0.78,-0.82,-0.85,-0.87,-0.89,-0.91,-0.93,-0.94,-0.95,-1.04,-1.04,-1.03
8,Pattern_Trend,-1,3,4,0.9,0.9,0.5,90,0,28,0.852463,-0.636667,-4.131050,-17.826667,0.9380,0.8832,0.7240,0.6944,0.5878,0.6564,0.7239,0.9879,1.1138,1.1484,1.1199,1.0201,0.8986,0.8510,0.8510,0.8460,0.7628,0.7715,0.8128,0.8014,0.8012,0.7983,0.8165,0.8502,-0.03,-0.06,-0.09,-0.10,-0.12,-0.15,-0.23,-0.31,-0.41,-0.51,-0.60,-0.67,-0.74,-0.80,-0.85,-0.88,-0.91,-0.96,-1.00,-1.03,-1.07,-1.23,-1.26,-1.27
21,Pattern_Trend,0,3,5,0.9,0.9,0.5,90,0,54,0.870529,-0.571250,-6.991425,-30.847500,0.9277,0.9933,1.0329,0.9524,0.8103,0.7955,0.8228,0.9299,0.9131,0.9294,0.9392,0.9014,0.8455,0.8061,0.8057,0.8341,0.7994,0.7955,0.8316,0.8183,0.8207,0.8584,0.8758,0.8537,-0.05,-0.09,-0.15,-0.18,-0.21,-0.25,-0.30,-0.36,-0.41,-0.48,-0.54,-0.59,-0.65,-0.70,-0.74,-0.77,-0.80,-0.83,-0.86,-0.88,-0.90,-0.98,-1.00,-0.99
10,Pattern_Trend,-1,3,6,0.9,0.9,0.5,90,0,59,0.890146,-0.500000,-6.481396,-29.500000,0.9230,0.9691,0.9965,0.9225,0.8205,0.8062,0.8377,1.0099,1.0158,1.0303,0.9720,0.9049,0.8199,0.7842,0.7892,0.8526,0.8348,0.8412,0.8695,0.8569,0.8480,0.8824,0.8963,0.8801,-0.04,-0.08,-0.13,-0.16,-0.19,-0.23,-0.28,-0.35,-0.41,-0.48,-0.54,-0.58,-0.62,-0.65,-0.67,-0.68,-0.69,-0.70,-0.72,-0.72,-0.73,-0.78,-0.79,-0.78
20,Pattern_Trend,0,3,4,0.9,0.9,0.5,90,0,34,0.930283,-0.479583,-2.370367,-16.305833,1.0448,1.0423,0.8948,0.8466,0.6961,0.7382,0.7931,0.9874,1.0869,1.1214,1.1229,1.0410,0.9485,0.9055,0.9034,0.9425,0.8623,0.8706,0.9060,0.8956,0.9032,0.9003,0.9222,0.9512,-0.03,-0.05,-0.07,-0.07,-0.06,-0.07,-0.12,-0.17,-0.24,-0.32,-0.39,-0.44,-0.51,-0.56,-0.61,-0.66,-0.70,-0.76,-0.80,-0.84,-0.88,-1.02,-1.06,-1.08
1,Pattern_Trend,-1,1,5,0.9,0.9,0.5,90,0,65,0.765437,-0.472083,-15.246562,-30.685417,0.8040,0.7266,0.7729,0.7579,0.7024,0.6940,0.7300,0.8450,0.8459,0.8730,0.8521,0.8162,0.7703,0.7410,0.7336,0.7259,0.7148,0.7185,0.7493,0.7376,0.7491,0.7755,0.7731,0.7618,-0.04,-0.07,-0.11,-0.14,-0.17,-0.21,-0.27,-0.33,-0.39,-0.45,-0.49,-0.54,-0.58,-0.61,-0.63,-0.64,-0.64,-0.66,-0.69,-0.70,-0.71,-0.76,-0.76,-0.74
5,Pattern_Trend,-1,2,5,0.9,0.9,0.5,90,0,63,0.784008,-0.457083,-13.607475,-28.796250,0.7964,0.7603,0.7793,0.7908,0.7337,0.7212,0.7630,0.8728,0.8804,0.8979,0.8742,0.8356,0.7866,0.7556,0.7477,0.7446,0.7275,0.7316,0.7630,0.7504,0.7603,0.7865,0.7846,0.7722,-0.04,-0.07,-0.11,-0.14,-0.17,-0.20,-0.25,-0.32,-0.38,-0.43,-0.47,-0.52,-0.55,-0.58,-0.60,-0.62,-0.63,-0.65,-0.67,-0.68,-0.69,-0.74,-0.74,-0.72
13,Pattern_Trend,0,1,5,0.9,0.9,0.5,90,0,76,0.844608,-0.435833,-11.809767,-33.123333,0.8960,0.8225,0.8555,0.8410,0.7832,0.7668,0.7982,0.9002,0.8949,0.9283,0.9303,0.8931,0.8507,0.8204,0.8099,0.8198,0.8055,0.8076,0.8344,0.8223,0.8351,0.8577,0.8562,0.8410,-0.03,-0.07,-0.10,-0.13,-0.15,-0.18,-0.23,-0.28,-0.33,-0.38,-0.43,-0.47,-

# Research parameter influence

In [8]:
param = 'Pattern_second_candle'

res = stat.groupby([param]).agg({'e_ratio_avg': 'mean',
                                 'pct_price_diff_avg': 'mean',
                                 'e_ratio_rank': 'mean', 
                                 'price_rank': 'mean', 
                                 'forecasts_num': 'sum'}).sort_values(param).reset_index()
res

,Pattern_second_candle,e_ratio_avg,pct_price_diff_avg,e_ratio_rank,price_rank,forecasts_num
0,0.5,0.854897,-0.304701,-168.318122,-286.847375,68130
1,0.6,0.850164,-0.288188,-150.008786,-258.243681,60370
2,0.7,0.849519,-0.281743,-133.779457,-224.315035,52007
3,0.8,0.839842,-0.299785,-109.302578,-205.237062,41617
4,0.9,0.871600,-0.322160,-72.804460,-170.303528,30776


# Plot result in file

In [11]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots()

x, y = param, 'e_ratio_avg'
ax.plot(res[x], res[y])
ax.set_xlabel(x)
ax.set_ylabel(y)
ax.set_title('_'.join(pattern) + '_' + ttype)

fig.savefig(f"{'_'.join(pattern)}'_'{ttype}_{param}", dpi=fig.dpi)

# Save new config data to config file

In [4]:
from config_updater import ConfigUpdater

ttype = 'sell'
pattern = ['Pattern', 'LinearReg']
work_timeframe = '15m'
higher_timeframe = '1h'
timeframe = f'{work_timeframe}_{higher_timeframe}'
        
optim_dict = {
                 'Pattern': {
                            'use_vol': [0], 
                            'window_low_bound': [1], 
                            'window_high_bound': [5],
                            'first_candle': [0.9],
                            'second_candle': [0.5],
                            'third_candle': [0.5]
                            },
                'Trend': {
                            'timeperiod': [28], 
                            'low_bound': [0]
                            }
             }
 
cu = ConfigUpdater(ttype, timeframe)
cu.config_update(optim_dict)